In [1]:
# MNIST classification - classifies hand written digits

# 2 layers:
# layer 0 - 784 nodes: input layer
# layer 1 - 10 nodes: hidden layer
# layer 2 - 10 nodes: output layer (represents 10 digits)

# training: forward propogation
# non linear filters used to increase complexity 
# probability function -  softmax

# weights leanring & optimization: backward propogation
# determine deviation of prediction, updates parameters (weights and baises) accordingly 

In [2]:
import numpy as np
import pandas as pd #for reading data
from matplotlib import pyplot as plt

In [5]:
# import training image data
data = pd.read_csv('train.csv')
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# transofrm into numpy array
data = np.array(data) 

# risk of overfitting - model may be perfectly trained to run on test data but not generalized for all data
# to fix - set aside dev data that we wont train with, cross validate with that data
r, f = data.shape # r - rows, f - features + label collumn 
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:f]
X_dev = X_dev/255.

data_train = data[1000:r].T
Y_train = data_train[0]
X_train = data_train[1:f]
X_train = X_train/255.

In [15]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
# initialize parameters

def init_params():
    W1 = np.random.rand(10, 784) - 0.5 # generate arrays from -0.5 to 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

# ReLu linear (filter) function
def ReLU(Z):
    return np.maximum(Z, 0) # array of elements greater than 0

# Softmax probability function
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

# forward propogation
def for_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2


# one hot encoding of Y matrix dealing with labels 
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

# derivative of ReLU
def ReLU_derive(Z):
    return (Z > 0) # linear fucntion --> derivative = 1, this means that if Z is a constant, we will return True , aka 1

# backward propogation 
def back_prop(Z1, A1, W1, Z2, A2, W2, X, Y):
    r = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / r * dZ2.dot(A1.T)
    db2 = 1 / r * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_derive(Z1)
    dW1 = 1 / r * dZ1.dot(X.T)
    db1 = 1 / r * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y)/ Y.size # calcultes avg correct, aka the accuracy 

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = for_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, W1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if (i % 10 == 0):
            print("Iteration: ", i)
            print("Accuracy: " , get_accuracy(get_predictions(A2), Y))
    return W1, b1, W1, b2
    

In [18]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 800, 0.1) 
# 87% accuracy with 800 training iterations

Iteration:  0
[0 8 8 ... 9 9 4] [6 2 9 ... 6 9 7]
Accuracy:  0.0707560975609756
Iteration:  10
[0 8 8 ... 1 9 0] [6 2 9 ... 6 9 7]
Accuracy:  0.16631707317073172
Iteration:  20
[0 3 9 ... 1 2 0] [6 2 9 ... 6 9 7]
Accuracy:  0.23612195121951218
Iteration:  30
[0 3 9 ... 1 2 0] [6 2 9 ... 6 9 7]
Accuracy:  0.2895609756097561
Iteration:  40
[0 3 9 ... 1 2 7] [6 2 9 ... 6 9 7]
Accuracy:  0.3407804878048781
Iteration:  50
[0 3 9 ... 1 2 7] [6 2 9 ... 6 9 7]
Accuracy:  0.38953658536585367
Iteration:  60
[7 3 9 ... 1 2 7] [6 2 9 ... 6 9 7]
Accuracy:  0.4330487804878049
Iteration:  70
[7 3 9 ... 1 2 7] [6 2 9 ... 6 9 7]
Accuracy:  0.4707073170731707
Iteration:  80
[6 3 9 ... 1 6 7] [6 2 9 ... 6 9 7]
Accuracy:  0.5057073170731707
Iteration:  90
[6 3 9 ... 1 6 7] [6 2 9 ... 6 9 7]
Accuracy:  0.5362439024390244
Iteration:  100
[6 3 9 ... 1 6 7] [6 2 9 ... 6 9 7]
Accuracy:  0.5649024390243902
Iteration:  110
[6 2 9 ... 1 9 7] [6 2 9 ... 6 9 7]
Accuracy:  0.5916829268292683
Iteration:  120
[6 2 9 .